# **Most_Active_Stocks**

The data were scraped from **Yahoo Finance** under the head of Most actice stocks.






## **Imported Usefull Libraries of python**



*  **BeautifulSoup** is a Python library used for web scraping or parsing HTML and XML documents. It provides a convenient way to extract data from HTML/XML files by parsing the document and creating a parse tree that can be traversed to extract data 

* **matplotlib.pyplot** is a Python module that provides a collection of functions for creating and manipulating plots and charts. It is part of the matplotlib library, which is a powerful plotting library that enables developers to create static, animated, and interactive visualizations in Python.

* **Plotly** is a web-based data visualization library that allows you to create interactive, publication-quality visualizations in Python. It provides an easy-to-use interface for creating a wide range of plot types, including scatter plots, line charts, bar charts, pie charts, and more. One of the unique features of Plotly is its ability to create interactive visualizations that can be explored and manipulated by the user.

*   **Seaborn** is a Python library based on matplotlib that provides a high-level interface for creating statistical visualizations. It provides a set of pre-defined plotting functions that can be used to create a wide range of statistical plots, such as scatter plots, line charts, bar charts, box plots, violin plots, and more. Seaborn also provides a number of customization options for modifying the appearance of plots.






In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
!pip install plotly==5.13.0
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


## **Header Scraping Method**
 Header scraping is a technique used to extract information about the headers (title, description, keywords, etc.) of a webpage.
 This method is an advanced scraping method. The code will automatically scrap the headers.

In [2]:
##      Generic scraping function
#Method Header

def scrap_table(url):
  req=requests.get(url)
  data=req.text
  soup=BeautifulSoup(data)

  raw_data = { }
  headers = []

  for i in soup.find_all("thead"):
    for j in i.find_all("th"):
      headers.append(j.text)
      raw_data[j.text]=[]

  for k in soup.find_all("tbody"):
    for l in k.find_all("tr"):
      for idx,cell in enumerate(l.find_all("td")):
        header_value=headers[idx]
        raw_data[header_value].append(cell.text)
  return pd.DataFrame(raw_data)

In [3]:
# defining a function for converting the numbers into a scalable number
def convert_to_scalablenumber(value):
    if "M" in value:
        return float(value.replace(",", "").replace("M", "")) * 1000000
    elif "B" in value:
        return float(value.replace(",", "").replace("B", "")) * 1000000000
    elif "T" in value:
        return float(value.replace(",", "").replace("T", "")) * 1000000000000
    else:
        return None

In [4]:
#converting string to float

def convert_col_to_float (df,columns):
  for i in columns:
    df[i]= pd.to_numeric(df[i].str.replace(",","").str.replace("%",""))
  return df

# **Heat Map of Most active Stocks** 

In [5]:
active_stocks_df=scrap_table("https://finance.yahoo.com/most-active?offset=0&count=200")
#converted the Datatype of the columns      
percentchange=active_stocks_df["% Change"].str.replace("+"," ",regex=False).str.replace("%"," ",regex=False).astype(float)
marketcap= active_stocks_df["Market Cap"].apply(convert_to_scalablenumber)


#Ploting a heatmap shoeing the Most active stock data
fig = px.treemap(active_stocks_df,
                 title = "Actice Stocks Heatmap (Size=Market Cap , Color=Percent Change",
                 path = ["Symbol"],
                 values =marketcap,
                 hover_name = "Name",
                 hover_data =['Price (Intraday)','% Change', 'Volume','Market Cap', 'PE Ratio (TTM)'],
                 color =percentchange,
                 color_continuous_scale="YlGnBu",
                 range_color=[-5,5])



#updating the layot of the chart
fig.update_layout(
    autosize=True,
    margin=dict(l=30,r=30,t=30,b=30),
    paper_bgcolor="white"
)

fig.show()


# **Bubble/scatter chart of Most active stocks**

---




In [6]:
#scraping the table from the web trought function made by above code 
active_stocks_df=scrap_table("https://finance.yahoo.com/most-active?offset=0&count=200")

#Chanding the datatype od the columns
percentage_Change =active_stocks_df["% Change"].str.replace("+"," ",regex=False).str.replace("%"," ",regex=False).astype(float)
price_inerday=active_stocks_df["Price (Intraday)"].str.replace(","," ").astype(float)
marketcap= active_stocks_df["Market Cap"].apply(convert_to_scalablenumber)


#ploting thr Bubble chart / scatter chart
fig2 = px.scatter(active_stocks_df, x=price_inerday,y=percentage_Change,
                  size=marketcap, color="Name",
                  hover_name="Name", size_max=60
                  )


#updating the layot of the chart
fig2.update_layout(
    autosize=True,
    margin=dict(l=30,r=30,t=30,b=30),
    paper_bgcolor="white"
)

fig2.show()

# **Rissing Stocks**




In [7]:

active_stocks_df=scrap_table("https://finance.yahoo.com/most-active?offset=0&count=200")

#changing the data type
active_stocks_df["% Change"]=active_stocks_df["% Change"].str.replace("+"," ",regex=False).str.replace("%"," ",regex=False).astype(float)

#   Filtering data 
#--------------filering  when % changing is greater then 0 it means it is rising  
rising_stock=active_stocks_df[active_stocks_df["% Change"]>0]

#Ploting Bar Chart for rising stocks
risingstock_chart = px.bar(rising_stock, y=rising_stock["% Change"], x=rising_stock["Name"],
             text_auto="0.2s",
             title="Active Rising Stocks",
             color="Name")

#updating the layout of charts
risingstock_chart.update_traces(textfont_size=12,
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False)
#-----Rising
risingstock_chart.show()
rising_stock






,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
0,TSLA,"Tesla, Inc.",202.07,+1.21,0.60,145.843M,169.547M,639.37B,54.91,
1,NVDA,NVIDIA Corporation,236.64,+29.10,14.02,111.8M,46.929M,589.707B,100.70,
2,AMD,"Advanced Micro Devices, Inc.",79.75,+3.14,4.10,76.391M,56.588M,128.586B,94.94,
6,INTC,Intel Corporation,25.61,+0.14,0.55,56.473M,40.71M,105.949B,13.20,
8,AMZN,"Amazon.com, Inc.",95.82,+0.03,0.03,48.467M,74.855M,981.896B,N/A,
9,AAPL,Apple Inc.,149.40,+0.49,0.33,48.394M,74.992M,2.364T,25.32,
16,RIG,Transocean Ltd.,6.52,+0.42,6.89,29.692M,22.387M,4.707B,N/A,
17,MSFT,Microsoft Corporation,254.77,+3.26,1.30,29.219M,31.208M,1.896T,28.31,
19,LAZR,"Luminar Technologies, Inc.",7.46,+0.06,0.81,28.046M,8.959M,2.706B,N/A,
20,BBD,Banco Bradesco S.A.,2.7100,+0.0300,1.12,27.745M,35.829M,28.84B,7.53,


# **Losing Stocks**

In [8]:

active_stocks_df=scrap_table("https://finance.yahoo.com/most-active?offset=0&count=200")

#changing the data type
active_stocks_df["% Change"]=active_stocks_df["% Change"].str.replace("+"," ",regex=False).str.replace("%"," ",regex=False).astype(float)
#-------------filtering when % change is lesser then 0 it maens it is losing
losing_stock=active_stocks_df[active_stocks_df["% Change"]<0]

#Ploting bar chart of losing stocks
losingstock_chart=px.bar(losing_stock,y=losing_stock["Name"],x=losing_stock["% Change"],
                         text_auto="0.2s",
                         title="Active Losing Stocks",
                         color="Name")
#updating the layout od the chart
losingstock_chart.update_traces(textfont_size=12,
                  textangle=0, 
                  textposition="outside", 
                  cliponaxis=False)

#----Printing the charts and table


#-----losing
losingstock_chart.show()
losing_stock


,Symbol,Name,Price (Intraday),Change,% Change,Volume,Avg Vol (3 month),Market Cap,PE Ratio (TTM),52 Week Range
3,LCID,"Lucid Group, Inc.",8.79,-1.19,-11.92,66.383M,34.137M,16.071B,9.06,
4,PLTR,Palantir Technologies Inc.,8.17,-0.19,-2.27,59.342M,40.434M,17.568B,N/A,
5,F,Ford Motor Company,12.07,-0.14,-1.15,57.439M,58.427M,48.113B,N/A,
7,NIO,NIO Inc.,9.79,-0.39,-3.83,49.723M,48.625M,16.354B,N/A,
10,AMC,"AMC Entertainment Holdings, Inc.",6.23,-0.03,-0.48,41.877M,33.305M,3.198B,N/A,
11,BABA,Alibaba Group Holding Limited,94.16,-0.62,-0.65,40.064M,21.203M,249.312B,125.55,
12,GOOGL,Alphabet Inc.,90.89,-0.76,-0.83,40.061M,36.619M,1.165T,19.93,
13,GOOG,Alphabet Inc.,91.07,-0.73,-0.80,32.424M,29.178M,1.165T,20.06,
14,U,Unity Software Inc.,31.83,-6.02,-15.90,31.424M,11.51M,12.829B,N/A,
15,MPW,"Medical Properties Trust, Inc.",11.14,-1.06,-8.69,28.992M,14.165M,6.662B,5.33,


#**Top 10 PE ratio stocks**


In [9]:
#scraping the data
active_stocks_df=scrap_table("https://finance.yahoo.com/most-active?offset=0&count=200")
#changing the data type and replace N/A value
active_stocks_df["PE Ratio (TTM)"]= active_stocks_df["PE Ratio (TTM)"].replace('N/A', 0).astype(float)

#top 10 PE Ratio (TTE)
top10_PEratio =active_stocks_df[["Name","PE Ratio (TTM)"]].sort_values(by=["PE Ratio (TTM)"],ascending = False)


top10_PEratio[:10]

,Name,PE Ratio (TTM)
11,Alibaba Group Holding Limited,125.55
1,NVIDIA Corporation,100.70
2,"Advanced Micro Devices, Inc.",94.94
0,"Tesla, Inc.",54.91
17,Microsoft Corporation,28.31
9,Apple Inc.,25.32
13,Alphabet Inc.,20.06
12,Alphabet Inc.,19.93
6,Intel Corporation,13.20
3,"Lucid Group, Inc.",9.06
